# Plusvalía estimada

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
import numpy as np

# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "alfa" in nombre and "queretaro" in nombre or ("df_junio_2023_queretaro" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_queretaro',
 'df_alfa_agosto_2024_queretaro',
 'df_alfa_marzo_2024_queretaro',
 'df_alfa_mayo_2024_queretaro',
 'df_junio_2023_queretaro']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Precio':'precio',}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")

# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['precio','precio']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)
print("Importadas")


Columnas renombradas en los DataFrames filtrados.
Importadas


# Plusvalía *ESTIMADA*

In [64]:
import re
# Función para extraer mes y año del nombre del DataFrame
def extraer_mes_y_ano(nombre):
    # Expresión regular para capturar tanto meses completos como abreviados
    match = re.search(r'(ene|feb|mar|abr|may|jun|jul|ago|sep|oct|nov|dic|enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)_(\d{4})', nombre)
    if match:
        mes = match.group(1).lower()  # Mes en texto
        ano = int(match.group(2))  # Año en formato numérico
        return mes, ano
    return None, None

# Diccionario de traducción de meses de texto a números
meses_dict = {
    'ene': 1, 'feb': 2, 'mar': 3, 'abr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'ago': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dic': 12,
    'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4, 'mayo': 5, 'junio': 6,
    'julio': 7, 'agosto': 8, 'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
}

# DataFrame para almacenar resultados
df_resultados = pd.DataFrame(columns=['precio_promedio', 'mes', 'año'])

for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame por su nombre en el entorno
    df = globals()[nombre_df]
    
    # Extraer mes y año del nombre del DataFrame
    mes_str, ano = extraer_mes_y_ano(nombre_df)
    
    # Verificar que se haya extraído el mes y el año correctamente
    if mes_str is not None and ano is not None:
        mes = meses_dict.get(mes_str)  # Convertir el nombre del mes en su número correspondiente
        
        # Calcular el promedio del precio y añadir columnas de mes y año
        precio_promedio = df['precio'].mean()  # Calcula el promedio del precio en el DataFrame actual
        df_resultados = pd.concat([df_resultados, pd.DataFrame({'precio_promedio': [precio_promedio], 'mes': [mes], 'año': [ano]})], ignore_index=True)

# Resultado final
print(df_resultados)


C:\Users\yoe11\AppData\Local\Temp\ipykernel_2112\1696226860.py:36: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



   precio_promedio mes   año
0     5.797868e+06   4  2024
1     6.978839e+06   8  2024
2     6.642340e+06   3  2024
3     6.212365e+06   5  2024
4     3.728279e+06   6  2023


In [65]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

# Preparar datos para el modelo
X = df_resultados[['año', 'mes']]
y = df_resultados['precio_promedio']

# Normalizar los datos de X
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Dividir los datos escalados en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal con datos escalados
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [68]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 4),
                            'mes': range(8, 12)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)

# Redondear las predicciones a números enteros
predicciones_enteros = np.round(predicciones).astype(int)

# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['precio_promedio'] = predicciones_enteros

nueva_conteo_pachuca = pd.concat([df_resultados, nuevas_fechas], ignore_index=True)
nueva_conteo_pachuca = nueva_conteo_pachuca.groupby('año')['precio_promedio'].median().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
nueva_conteo_pachuca.head()

c:\Users\yoe11\anaconda3\envs\newst\Lib\site-packages\sklearn\base.py:486: UserWarning:

X has feature names, but LinearRegression was fitted without feature names



,año,precio_promedio
0,2023,4.159526e+09
1,2024,4.163198e+09
2,2025,4.167407e+09
3,2026,4.169466e+09


In [69]:
# Datos
año = nueva_conteo_pachuca['año']
promedio = nueva_conteo_pachuca['precio_promedio']

# Crear una función para formatear los valores grandes en palabras en español
def formatear_numero(valor):
    if valor >= 1_000_000_000_000:
        return f'{valor / 1_000_000_000:.2f} billones'
    elif valor >= 1_000_000_000:
        return f'{valor / 1_000_000_000:.0f} mil millones'  # Mil millones
    elif valor >= 1_000_000:
        return f'{valor / 1_000_000:.0f} millones'  # Millones
    elif valor >= 1_000:
        return f'{valor / 1_000:.0f} mil'  # Mil
    else:
        return str(valor)
# Configurar los valores y el texto del eje Y
tickvals = [4150000000000, 4160000000000, 4170000000000]
ticktext = [formatear_numero(val) for val in tickvals]

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=año,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for a, p in zip(año, promedio):
    fig.add_annotation(
        x=a,
        y=p,
        text=f"${p:,.2f}",  # Formatear el valor del promedio como moneda
        showarrow=False,
        font=dict(color='black', size=12),
        xshift=0,
        yshift=17,
        textangle=0
    )
# Actualizar diseño
fig.update_layout(
    #title='Plusvalía estimada',   
    yaxis=dict(
        title="Monto",
        gridwidth=1,  # Ancho de las líneas de la cuadrícula
        gridcolor='#dddcda', 

    ),
    xaxis=dict(
        gridcolor='#dddcda', 
        tickmode='array',
        tickvals=año,
        ticktext=año
    ),
    plot_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=10, r=10, t=10, b=10)

)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_plusvaliaestimada', carpeta='graficas')
fig.show()

# Plusvalía *PROMEDIO*

In [59]:
# Ordenar el DataFrame por año
df_resultados = df_resultados.sort_values(by='año').reset_index(drop=True)

# Calcular el cambio porcentual anual en la columna 'precio_promedio'
df_resultados['incremento_anual'] = df_resultados['precio_promedio'].pct_change().round(2) * 100

# Rellenar con 0 en el primer año, ya que no tiene año anterior para calcular el cambio
df_resultados['incremento_anual'].fillna(0, inplace=True)

# Mostrar el resultado
df_resultados.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_2112\3103264345.py:8: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





,precio_promedio,mes,año,incremento_anual
0,3.728279e+06,6,2023,0.0
1,5.797868e+06,4,2024,56.0
2,6.978839e+06,8,2024,20.0
3,6.642340e+06,3,2024,-5.0
4,6.212365e+06,5,2024,-6.0


In [63]:
dfsg = df_resultados[df_resultados['año']==2024]
dfsg['incremento_anual'].mean()

np.float64(16.25)

In [60]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

# Preparar datos para el modelo
X = df_resultados[['año', 'mes']]
y = df_resultados['incremento_anual']

# Normalizar los datos de X
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Dividir los datos escalados en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal con datos escalados
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [62]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2024, 4),
                            'mes': range(9, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
                                  
nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 
# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas[['año', 'mes']])  # Asegurarse de que se están pasando las columnas correctas

# Redondear las predicciones a números enteros
predicciones_enteros = np.round(predicciones).astype(int)

# Limitar las predicciones a un máximo de 100%
predicciones_enteros = np.where(predicciones_enteros > 100, 100, predicciones_enteros)

# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['incremento_anual'] = predicciones_enteros

nueva_conteo_pachuca = pd.concat([df_resultados, nuevas_fechas], ignore_index=True)
nueva_conteo_pachuca = nueva_conteo_pachuca.groupby('año')['incremento_anual'].mean().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
nueva_conteo_pachuca.head()

c:\Users\yoe11\anaconda3\envs\newst\Lib\site-packages\sklearn\base.py:486: UserWarning:

X has feature names, but LinearRegression was fitted without feature names



,año,incremento_anual
0,2023,0.000
1,2024,58.125
2,2025,100.000
3,2026,100.000


In [ ]:
g_scatt_plusvaliaestimada_porcentual

# MEDIO

# Plusvalía *ESTIMADA* [Mensual]

In [3]:
import re
# Función para extraer mes y año del nombre del DataFrame
def extraer_mes_y_ano(nombre):
    # Expresión regular para capturar tanto meses completos como abreviados
    match = re.search(r'(ene|feb|mar|abr|may|jun|jul|ago|sep|oct|nov|dic|enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)_(\d{4})', nombre)
    if match:
        mes = match.group(1).lower()  # Mes en texto
        ano = int(match.group(2))  # Año en formato numérico
        return mes, ano
    return None, None

# Diccionario de traducción de meses de texto a números
meses_dict = {
    'ene': 1, 'feb': 2, 'mar': 3, 'abr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'ago': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dic': 12,
    'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4, 'mayo': 5, 'junio': 6,
    'julio': 7, 'agosto': 8, 'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
}

# DataFrame para almacenar resultados
df_resultados = pd.DataFrame(columns=['precio_promedio', 'mes', 'año'])

for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame por su nombre en el entorno
    df = globals()[nombre_df]
    
    # Extraer mes y año del nombre del DataFrame
    mes_str, ano = extraer_mes_y_ano(nombre_df)
    
    # Verificar que se haya extraído el mes y el año correctamente
    if mes_str is not None and ano is not None:
        mes = meses_dict.get(mes_str)  # Convertir el nombre del mes en su número correspondiente
        
        # Calcular el promedio del precio y añadir columnas de mes y año
        precio_promedio = df['precio'].mean()  # Calcula el promedio del precio en el DataFrame actual
        df_resultados = pd.concat([df_resultados, pd.DataFrame({'precio_promedio': [precio_promedio], 'mes': [mes], 'año': [ano]})], ignore_index=True)

# Resultado final
print(df_resultados)

   precio_promedio mes   año
0     5.797868e+06   4  2024
1     6.978839e+06   8  2024
2     6.642340e+06   3  2024
3     6.212365e+06   5  2024
4     3.728279e+06   6  2023


C:\Users\yoe11\AppData\Local\Temp\ipykernel_3176\3462637469.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_resultados = pd.concat([df_resultados, pd.DataFrame({'precio_promedio': [precio_promedio], 'mes': [mes], 'año': [ano]})], ignore_index=True)


In [4]:
df_resultados = df_resultados[~df_resultados['año'].isin([2022, 2023]) ]#& (df_resultados['mes'] != 4)]
df_resultados

,precio_promedio,mes,año
0,5.797868e+06,4,2024
1,6.978839e+06,8,2024
2,6.642340e+06,3,2024
3,6.212365e+06,5,2024


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

# Preparar datos para el modelo
X = df_resultados[['año', 'mes']]
y = df_resultados['precio_promedio']

# Normalizar los datos de X
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Dividir los datos escalados en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal con datos escalados
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [9]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2024, 2),
                            'mes': range(1, 3)})
fechas_2024 = pd.DataFrame({'año': np.repeat(2024, 2),
                            'mes': range(6, 8)})
fechas_20244= pd.DataFrame({'año': np.repeat(2024, 4),
                            'mes': range(9, 13)})
# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2023,fechas_2024,fechas_20244, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)

# Redondear las predicciones a números enteros
predicciones_enteros = np.round(predicciones).astype(int)

# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['precio_promedio'] = predicciones_enteros

nueva_conteo_pachuca = pd.concat([df_resultados, nuevas_fechas], ignore_index=True)
#nueva_conteo_pachuca = nueva_conteo_pachuca.groupby('año')['precio_promedio'].median().reset_index()
nueva_conteo_pachuca = nueva_conteo_pachuca.groupby(['año', 'mes'])['precio_promedio'].median().reset_index()

# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_todos = nueva_conteo_pachuca.copy()
nueva_conteo_pachuca

c:\Users\yoe11\anaconda3\envs\newst\Lib\site-packages\sklearn\base.py:486: UserWarning:

X has feature names, but LinearRegression was fitted without feature names



,año,mes,precio_promedio
0,2024,1,5.357573e+06
1,2024,2,4.282634e+06
2,2024,3,6.642340e+06
3,2024,4,5.797868e+06
4,2024,5,6.212365e+06
5,2024,6,-1.712100e+04
6,2024,7,-1.092059e+06
7,2024,8,6.978839e+06
8,2024,9,-3.241937e+06
9,2024,10,-4.316875e+06


In [10]:
meses_dict_invertido = {v: k for k, v in meses_dict.items()}
nueva_conteo_pachuca['label'] = nueva_conteo_pachuca['mes'].apply(lambda x: meses_dict_invertido[x]) + '-' + nueva_conteo_pachuca['año'].astype(str)
nueva_conteo_pachuca = nueva_conteo_pachuca.sort_values(by=['año', 'mes'])
nueva_conteo_pachuca

,año,mes,precio_promedio,label
0,2024,1,5.357573e+06,enero-2024
1,2024,2,4.282634e+06,febrero-2024
2,2024,3,6.642340e+06,marzo-2024
3,2024,4,5.797868e+06,abril-2024
4,2024,5,6.212365e+06,mayo-2024
5,2024,6,-1.712100e+04,junio-2024
6,2024,7,-1.092059e+06,julio-2024
7,2024,8,6.978839e+06,agosto-2024
8,2024,9,-3.241937e+06,septiembre-2024
9,2024,10,-4.316875e+06,octubre-2024


In [11]:
# Datos
año = nueva_conteo_pachuca['label']
promedio = nueva_conteo_pachuca['precio_promedio']

# Crear una función para formatear los valores grandes en palabras en español
def formatear_numero(valor):
    if valor >= 1_000_000_000_000:
        return f'{valor / 1_000_000_000:.2f} billones'
    elif valor >= 1_000_000_000:
        return f'{valor / 1_000_000_000:.0f} mil millones'  # Mil millones
    elif valor >= 1_000_000:
        return f'{valor / 1_000_000:.0f} millones'  # Millones
    elif valor >= 1_000:
        return f'{valor / 1_000:.0f} mil'  # Mil
    else:
        return str(valor)
# Configurar los valores y el texto del eje Y
tickvals = [4150000000000, 4160000000000, 4170000000000]
ticktext = [formatear_numero(val) for val in tickvals]

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=año,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for a, p in zip(año, promedio):
    fig.add_annotation(
        x=a,
        y=p,
        text=f"${p:,.2f}",  # Formatear el valor del promedio como moneda
        showarrow=False,
        font=dict(color='black', size=12),
        xshift=0,
        yshift=17,
        textangle=0
    )
# Actualizar diseño
fig.update_layout(
    #title='Plusvalía estimada',   
    yaxis=dict(
        title="Monto",
        gridwidth=1,  # Ancho de las líneas de la cuadrícula
        gridcolor='#dddcda', 

    ),
    xaxis=dict(
        gridcolor='#dddcda', 
        tickmode='array',
        tickvals=año,
        ticktext=año
    ),
    plot_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=10, r=10, t=10, b=10)

)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_plusvaliaestimada', carpeta='graficas')
fig.show()

# LAMUDI

In [3]:
import pandas as pd

# Crear un DataFrame para los datos de precio promedio de casas
df_casas = pd.DataFrame([
    {"label":"Oct 23","mes":"Oct","año":"2023","value":4350000},
    {"label":"Nov 23","mes":"Nov","año":"2023","value":4350000},
    {"label":"Dic 23","mes":"Dic","año":"2023","value":4400000},
    {"label":"Ene 24","mes":"Ene","año":"2024","value":4480000},
    {"label":"Feb 24","mes":"Feb","año":"2024","value":4500000},
    {"label":"Mar 24","mes":"Mar","año":"2024","value":4500000},
    {"label":"Abr 24","mes":"Abr","año":"2024","value":4500000},
    {"label":"May 24","mes":"May","año":"2024","value":4614000},
    {"label":"Jun 24","mes":"Jun","año":"2024","value":4830000},
    {"label":"Jul 24","mes":"Jul","año":"2024","value":4915000},
    {"label":"Ago 24","mes":"Ago","año":"2024","value":4950000},
    {"label":"Sept 24","mes":"Sept","año":"2024","value":5090000},
    {"label":"Oct 24","mes":"Oct","año":"2024","value":5100000}
])

# Crear un DataFrame para los datos de precio promedio de departamentos
df_departamentos = pd.DataFrame([
    {"label":"Oct 23","mes":"Oct","año":"2023","value":3390000},
    {"label":"Nov 23","mes":"Nov","año":"2023","value":3400000},
    {"label":"Dic 23","mes":"Dic","año":"2023","value":3400000},
    {"label":"Ene 24","mes":"Ene","año":"2024","value":3400000},
    {"label":"Feb 24","mes":"Feb","año":"2024","value":3450000},
    {"label":"Mar 24","mes":"Mar","año":"2024","value":3500000},
    {"label":"Abr 24","mes":"Abr","año":"2024","value":3500000},
    {"label":"May 24","mes":"May","año":"2024","value":3550000},
    {"label":"Jun 24","mes":"Jun","año":"2024","value":3568135},
    {"label":"Jul 24","mes":"Jul","año":"2024","value":3590000},
    {"label":"Ago 24","mes":"Ago","año":"2024","value":3582140},
    {"label":"Sept 24","mes":"Sept","año":"2024","value":3350000},
    {"label":"Oct 24","mes":"Oct","año":"2024","value":3380000}
])

# Calcular el promedio de los datos por label único, incluyendo mes y año
df_promedio = pd.merge(df_casas.groupby(['mes', 'año'])['value'].mean().reset_index(),
                       df_departamentos.groupby(['mes', 'año'])['value'].mean().reset_index(),
                       on=['mes', 'año'], how='inner')

df_promedio['promedio'] = (df_promedio['value_x'] + df_promedio['value_y']) / 2

# Mostrar el DataFrame
print(df_promedio)

     mes   año    value_x    value_y   promedio
0    Abr  2024  4500000.0  3500000.0  4000000.0
1    Ago  2024  4950000.0  3582140.0  4266070.0
2    Dic  2023  4400000.0  3400000.0  3900000.0
3    Ene  2024  4480000.0  3400000.0  3940000.0
4    Feb  2024  4500000.0  3450000.0  3975000.0
5    Jul  2024  4915000.0  3590000.0  4252500.0
6    Jun  2024  4830000.0  3568135.0  4199067.5
7    Mar  2024  4500000.0  3500000.0  4000000.0
8    May  2024  4614000.0  3550000.0  4082000.0
9    Nov  2023  4350000.0  3400000.0  3875000.0
10   Oct  2023  4350000.0  3390000.0  3870000.0
11   Oct  2024  5100000.0  3380000.0  4240000.0
12  Sept  2024  5090000.0  3350000.0  4220000.0


In [4]:
df_promedio['label'] = df_promedio['mes'] + '-' + df_promedio['año']
df_promedio['mes_numero'] = df_promedio['mes'].map({'Ene': '1', 'Feb': '2', 'Mar': '3', 'Abr': '4', 'May': '5', 'Jun': '6', 'Jul': '7', 'Ago': '8', 'Sept': '9', 'Oct': '10', 'Nov': '11', 'Dic': '12'})
df_promedio = df_promedio.sort_values(by=['año', 'mes_numero']).reset_index(drop=True)
df_promedio

,mes,año,value_x,value_y,promedio,label,mes_numero
0,Oct,2023,4350000.0,3390000.0,3870000.0,Oct-2023,10
1,Nov,2023,4350000.0,3400000.0,3875000.0,Nov-2023,11
2,Dic,2023,4400000.0,3400000.0,3900000.0,Dic-2023,12
3,Ene,2024,4480000.0,3400000.0,3940000.0,Ene-2024,1
4,Oct,2024,5100000.0,3380000.0,4240000.0,Oct-2024,10
5,Feb,2024,4500000.0,3450000.0,3975000.0,Feb-2024,2
6,Mar,2024,4500000.0,3500000.0,4000000.0,Mar-2024,3
7,Abr,2024,4500000.0,3500000.0,4000000.0,Abr-2024,4
8,May,2024,4614000.0,3550000.0,4082000.0,May-2024,5
9,Jun,2024,4830000.0,3568135.0,4199067.5,Jun-2024,6


In [5]:
df_promedio['mes_numero'] = df_promedio['mes_numero'].astype(int)
df_promedio['año'] = df_promedio['año'].astype(int)
df_promedio = df_promedio.sort_values(by=['año', 'mes_numero']).reset_index(drop=True)

df_promedio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   mes         13 non-null     object 
 1   año         13 non-null     int64  
 2   value_x     13 non-null     float64
 3   value_y     13 non-null     float64
 4   promedio    13 non-null     float64
 5   label       13 non-null     object 
 6   mes_numero  13 non-null     int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 860.0+ bytes


In [6]:
# Datos
año = df_promedio['label']
promedio = df_promedio['promedio']

# Crear una función para formatear los valores grandes en palabras en español
def formatear_numero(valor):
    if valor >= 1_000_000_000_000:
        return f'{valor / 1_000_000_000:.2f} billones'
    elif valor >= 1_000_000_000:
        return f'{valor / 1_000_000_000:.0f} mil millones'  # Mil millones
    elif valor >= 1_000_000:
        return f'{valor / 1_000_000:.0f} millones'  # Millones
    elif valor >= 1_000:
        return f'{valor / 1_000:.0f} mil'  # Mil
    else:
        return str(valor)
# Configurar los valores y el texto del eje Y
tickvals = [2500000000,3000000000,3200000000]
ticktext = [formatear_numero(val) for val in tickvals]

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=año,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for a, p in zip(año, promedio):
    fig.add_annotation(
        x=a,
        y=p,
        text=f"${p:,.2f}",  # Formatear el valor del promedio como moneda
        showarrow=False,
        font=dict(color='black', size=12),
        xshift=0,
        yshift=17,
        textangle=0
    )
# Actualizar diseño
fig.update_layout(
    #title='Plusvalía estimada',   
    yaxis=dict(
        title="Monto",
        gridwidth=1,  # Ancho de las líneas de la cuadrícula
        gridcolor='#dddcda', 

    ),
    xaxis=dict(
        gridcolor='#dddcda', 
        tickmode='array',
        tickvals=año,
        ticktext=año
    ),
    plot_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=10, r=10, t=10, b=10)
)
fig.show()

In [7]:
df_promedio['cambio_porcentual'] = df_promedio['promedio'].pct_change()
df_promedio

,mes,año,value_x,value_y,promedio,label,mes_numero,cambio_porcentual
0,Oct,2023,4350000.0,3390000.0,3870000.0,Oct-2023,10,NaN
1,Nov,2023,4350000.0,3400000.0,3875000.0,Nov-2023,11,0.001292
2,Dic,2023,4400000.0,3400000.0,3900000.0,Dic-2023,12,0.006452
3,Ene,2024,4480000.0,3400000.0,3940000.0,Ene-2024,1,0.010256
4,Feb,2024,4500000.0,3450000.0,3975000.0,Feb-2024,2,0.008883
5,Mar,2024,4500000.0,3500000.0,4000000.0,Mar-2024,3,0.006289
6,Abr,2024,4500000.0,3500000.0,4000000.0,Abr-2024,4,0.000000
7,May,2024,4614000.0,3550000.0,4082000.0,May-2024,5,0.020500
8,Jun,2024,4830000.0,3568135.0,4199067.5,Jun-2024,6,0.028679
9,Jul,2024,4915000.0,3590000.0,4252500.0,Jul-2024,7,0.012725


In [8]:
tasa_cambio = df_promedio['cambio_porcentual'].mean()

# Aplica la misma tasa de cambio para estimar cada mes de 2024, 2025, y 2026
estimaciones = {}
ultimo_promedio = df_promedio[df_promedio['año'] == 2023]['promedio'].iloc[-1]  # Obtener el último promedio de 2023
for año in [2024, 2025, 2026]:
    for mes in range(1, 13):  # Itera sobre los 12 meses del año
        if año == 2024 and mes == 1:
            estimaciones[(mes, año)] = ultimo_promedio * (1 + tasa_cambio)
        else:
            if (mes-1, año) in estimaciones:
                estimaciones[(mes, año)] = estimaciones[(mes-1, año)] * (1 + tasa_cambio)
            else:
                estimaciones[(mes, año)] = ultimo_promedio * (1 + tasa_cambio)

# Imprime las estimaciones para cada mes de 2024, 2025, y 2026
for año in [2024, 2025, 2026]:
    for mes in range(1, 13):
        print(f"Estimación de registros en {mes}/{año}: {round(estimaciones[(mes, año)])}")

Estimación de registros en 1/2024: 3929967
Estimación de registros en 2/2024: 3960165
Estimación de registros en 3/2024: 3990595
Estimación de registros en 4/2024: 4021259
Estimación de registros en 5/2024: 4052158
Estimación de registros en 6/2024: 4083294
Estimación de registros en 7/2024: 4114670
Estimación de registros en 8/2024: 4146287
Estimación de registros en 9/2024: 4178147
Estimación de registros en 10/2024: 4210252
Estimación de registros en 11/2024: 4242603
Estimación de registros en 12/2024: 4275204
Estimación de registros en 1/2025: 3929967
Estimación de registros en 2/2025: 3960165
Estimación de registros en 3/2025: 3990595
Estimación de registros en 4/2025: 4021259
Estimación de registros en 5/2025: 4052158
Estimación de registros en 6/2025: 4083294
Estimación de registros en 7/2025: 4114670
Estimación de registros en 8/2025: 4146287
Estimación de registros en 9/2025: 4178147
Estimación de registros en 10/2025: 4210252
Estimación de registros en 11/2025: 4242603
Estima

In [9]:
df_promedio_proyección = df_promedio.copy()

# Crear un DataFrame temporal para agregar los nuevos registros
df_temporal = pd.DataFrame([
    {'mes': 'Nov', 'año': 2024, 'promedio': 4242603, 'label': 'Nov-2024', 'mes_numero': 11},
    {'mes': 'Dic', 'año': 2024, 'promedio': 4275204, 'label': 'Dic-2024', 'mes_numero': 12},
    {'mes': 'Ene', 'año': 2025, 'promedio': 3929967, 'label': 'Ene-2025', 'mes_numero': 1},
    {'mes': 'Feb', 'año': 2025, 'promedio': 3960165, 'label': 'Feb-2025', 'mes_numero': 2},
    {'mes': 'Mar', 'año': 2025, 'promedio': 3990595, 'label': 'Mar-2025', 'mes_numero': 3},
    {'mes': 'Abr', 'año': 2025, 'promedio': 4021259, 'label': 'Abr-2025', 'mes_numero': 4},
    {'mes': 'May', 'año': 2025, 'promedio': 4052158, 'label': 'May-2025', 'mes_numero': 5},
    {'mes': 'Jun', 'año': 2025, 'promedio': 4083294, 'label': 'Jun-2025', 'mes_numero': 6},
    {'mes': 'Jul', 'año': 2025, 'promedio': 4114670, 'label': 'Jul-2025', 'mes_numero': 7},
    {'mes': 'Ago', 'año': 2025, 'promedio': 4146287, 'label': 'Ago-2025', 'mes_numero': 8},
    {'mes': 'Sep', 'año': 2025, 'promedio': 4178147, 'label': 'Sep-2025', 'mes_numero': 9},
    {'mes': 'Oct', 'año': 2025, 'promedio': 4210252, 'label': 'Oct-2025', 'mes_numero': 10},
    {'mes': 'Nov', 'año': 2025, 'promedio': 4242603, 'label': 'Nov-2025', 'mes_numero': 11},
    {'mes': 'Dic', 'año': 2025, 'promedio': 4275204, 'label': 'Dic-2025', 'mes_numero': 12},
    {'mes': 'Ene', 'año': 2026, 'promedio': 3929967, 'label': 'Ene-2026', 'mes_numero': 1},
    {'mes': 'Feb', 'año': 2026, 'promedio': 3960165, 'label': 'Feb-2026', 'mes_numero': 2},
    {'mes': 'Mar', 'año': 2026, 'promedio': 3990595, 'label': 'Mar-2026', 'mes_numero': 3},
    {'mes': 'Abr', 'año': 2026, 'promedio': 4021259, 'label': 'Abr-2026', 'mes_numero': 4},
    {'mes': 'May', 'año': 2026, 'promedio': 4052158, 'label': 'May-2026', 'mes_numero': 5},
    {'mes': 'Jun', 'año': 2026, 'promedio': 4083294, 'label': 'Jun-2026', 'mes_numero': 6},
    {'mes': 'Jul', 'año': 2026, 'promedio': 4114670, 'label': 'Jul-2026', 'mes_numero': 7},
    {'mes': 'Ago', 'año': 2026, 'promedio': 4146287, 'label': 'Ago-2026', 'mes_numero': 8},
    {'mes': 'Sep', 'año': 2026, 'promedio': 4178147, 'label': 'Sep-2026', 'mes_numero': 9},
    {'mes': 'Oct', 'año': 2026, 'promedio': 4210252, 'label': 'Oct-2026', 'mes_numero': 10},
    {'mes': 'Nov', 'año': 2026, 'promedio': 4242603, 'label': 'Nov-2026', 'mes_numero': 11},
    {'mes': 'Dic', 'año': 2026, 'promedio': 4275204, 'label': 'Dic-2026', 'mes_numero': 12}
], columns=['mes', 'año', 'promedio', 'label', 'mes_numero'])

# Concatenar el DataFrame temporal al DataFrame original
df_promedio_proyección = pd.concat([df_promedio_proyección, df_temporal], ignore_index=True)

df_promedio_proyección

,mes,año,value_x,value_y,promedio,label,mes_numero,cambio_porcentual
0,Oct,2023,4350000.0,3390000.0,3870000.0,Oct-2023,10,NaN
1,Nov,2023,4350000.0,3400000.0,3875000.0,Nov-2023,11,0.001292
2,Dic,2023,4400000.0,3400000.0,3900000.0,Dic-2023,12,0.006452
3,Ene,2024,4480000.0,3400000.0,3940000.0,Ene-2024,1,0.010256
4,Feb,2024,4500000.0,3450000.0,3975000.0,Feb-2024,2,0.008883
5,Mar,2024,4500000.0,3500000.0,4000000.0,Mar-2024,3,0.006289
6,Abr,2024,4500000.0,3500000.0,4000000.0,Abr-2024,4,0.000000
7,May,2024,4614000.0,3550000.0,4082000.0,May-2024,5,0.020500
8,Jun,2024,4830000.0,3568135.0,4199067.5,Jun-2024,6,0.028679
9,Jul,2024,4915000.0,3590000.0,4252500.0,Jul-2024,7,0.012725


In [10]:
# Datos
año = df_promedio_proyección['label']
promedio = df_promedio_proyección['promedio']

# Crear una función para formatear los valores grandes en palabras en español
def formatear_numero(valor):
    if valor >= 1_000_000_000_000:
        return f'{valor / 1_000_000_000:.2f} billones'
    elif valor >= 1_000_000_000:
        return f'{valor / 1_000_000_000:.0f} mil millones'  # Mil millones
    elif valor >= 1_000_000:
        return f'{valor / 1_000_000:.0f} millones'  # Millones
    elif valor >= 1_000:
        return f'{valor / 1_000:.0f} mil'  # Mil
    else:
        return str(valor)
# Configurar los valores y el texto del eje Y
tickvals = [2500000000,3000000000,3200000000]
ticktext = [formatear_numero(val) for val in tickvals]

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=año,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for a, p in zip(año, promedio):
    fig.add_annotation(
        x=a,
        y=p,
        text=f"${p:,.2f}",  # Formatear el valor del promedio como moneda
        showarrow=False,
        font=dict(color='black', size=12),
        xshift=0,
        yshift=17,
        textangle=0
    )
# Actualizar diseño
fig.update_layout(
    #title='Plusvalía estimada',   
    yaxis=dict(
        title="Monto",
        gridwidth=1,  # Ancho de las líneas de la cuadrícula
        gridcolor='#dddcda', 

    ),
    xaxis=dict(
        gridcolor='#dddcda', 
        tickmode='array',
        tickvals=año,
        ticktext=año
    ),
    plot_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=10, r=10, t=10, b=10)
)
fig.show()

In [11]:
# Crear un nuevo DataFrame para las proyecciones
nuevos_meses = []
ultimo_promedio = df_promedio['promedio'].iloc[-1]  # Obtener el último promedio de la tabla existente

# Proyectar hasta diciembre de 2026
for año in range(2024, 2027):
    for mes in range(1, 13):  # Iterar sobre los 12 meses
        # Calcular el nuevo promedio usando el cambio porcentual del mes anterior
        if mes == 1 and año == 2024:
            # Para enero de 2024, usar el último promedio de 2023
            nuevo_promedio = ultimo_promedio * (1 + df_promedio['cambio_porcentual'].iloc[-1] if not pd.isna(df_promedio['cambio_porcentual'].iloc[-1]) else 0)
        else:
            # Para otros meses, usar el promedio del mes anterior
            if mes == 1:
                # Si es enero, usar el promedio de diciembre del año anterior
                nuevo_promedio = ultimo_promedio * (1 + df_promedio['cambio_porcentual'].iloc[-1] if not pd.isna(df_promedio['cambio_porcentual'].iloc[-1]) else 0)
            else:
                # Usar el cambio porcentual del mes anterior
                if not pd.isna(df_promedio['cambio_porcentual'].iloc[mes - 2]):
                    nuevo_promedio = ultimo_promedio * (1 + df_promedio['cambio_porcentual'].iloc[mes - 2])
                else:
                    nuevo_promedio = ultimo_promedio  # Si no hay cambio, mantener el último promedio

        # Agregar el nuevo mes y año al DataFrame
        nuevos_meses.append({'mes': mes, 'año': año, 'promedio': nuevo_promedio})
        ultimo_promedio = nuevo_promedio  # Actualizar el último promedio

# Convertir la lista a un DataFrame
df_proyecciones = pd.DataFrame(nuevos_meses)

# Crear una etiqueta para el mes
df_proyecciones['label'] = df_proyecciones['mes'].astype(str) + '-' + df_proyecciones['año'].astype(str)

# Mostrar el DataFrame de proyecciones
print(df_proyecciones)

    mes   año      promedio    label
0     1  2024  4.260095e+06   1-2024
1     2  2024  4.260095e+06   2-2024
2     3  2024  4.265599e+06   3-2024
3     4  2024  4.293119e+06   4-2024
4     5  2024  4.337151e+06   5-2024
5     6  2024  4.375679e+06   6-2024
6     7  2024  4.403199e+06   7-2024
7     8  2024  4.403199e+06   8-2024
8     9  2024  4.493464e+06   9-2024
9    10  2024  4.622332e+06  10-2024
10   11  2024  4.681151e+06  11-2024
11   12  2024  4.696089e+06  12-2024
12    1  2025  4.718345e+06   1-2025
13    2  2025  4.718345e+06   2-2025
14    3  2025  4.724441e+06   3-2025
15    4  2025  4.754921e+06   4-2025
16    5  2025  4.803690e+06   5-2025
17    6  2025  4.846362e+06   6-2025
18    7  2025  4.876842e+06   7-2025
19    8  2025  4.876842e+06   8-2025
20    9  2025  4.976818e+06   9-2025
21   10  2025  5.119547e+06  10-2025
22   11  2025  5.184693e+06  11-2025
23   12  2025  5.201238e+06  12-2025
24    1  2026  5.225888e+06   1-2026
25    2  2026  5.225888e+06   2-2026
2

In [12]:
df_proyecciones_24 = df_proyecciones[df_proyecciones['año'] == 2024]

In [18]:
# Datos
año = df_proyecciones_24['label']
promedio = df_proyecciones_24['promedio']

# Crear una función para formatear los valores grandes en palabras en español
def formatear_numero(valor):
    if valor >= 1_000_000_000_000:
        return f'{valor / 1_000_000_000:.2f} billones'
    elif valor >= 1_000_000_000:
        return f'{valor / 1_000_000_000:.0f} mil millones'  # Mil millones
    elif valor >= 1_000_000:
        return f'{valor / 1_000_000:.0f} millones'  # Millones
    elif valor >= 1_000:
        return f'{valor / 1_000:.0f} mil'  # Mil
    else:
        return str(valor)
# Configurar los valores y el texto del eje Y
tickvals = [2500000000,3000000000,3200000000]
ticktext = [formatear_numero(val) for val in tickvals]

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=año,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for a, p in zip(año, promedio):
    fig.add_annotation(
        x=a,
        y=p,
        text=f"${p:,.2f}",  # Formatear el valor del promedio como moneda
        showarrow=False,
        font=dict(color='black', size=12),
        xshift=0,
        yshift=17,
        textangle=0
    )
# Actualizar diseño
fig.update_layout(
    #title='Plusvalía estimada',   
    yaxis=dict(
        title="Monto",
        gridwidth=1,  # Ancho de las líneas de la cuadrícula
        gridcolor='#dddcda', 

    ),
    xaxis=dict(
        gridcolor='#dddcda', 
        tickmode='array',
        tickvals=año,
        ticktext=año
    ),
    plot_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=10, r=10, t=10, b=10)
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_plusvaliaestimada_mes24', carpeta='assets/graficas')
fig.show()

In [14]:
nueva_conteo_pachuca = df_proyecciones.groupby('año')['promedio'].mean().reset_index()
nueva_conteo_pachuca

,año,promedio
0,2024,4.424264e+06
1,2025,4.900174e+06
2,2026,5.427276e+06


In [17]:
# Datos
año = nueva_conteo_pachuca['año']
promedio = nueva_conteo_pachuca['promedio']

# Crear una función para formatear los valores grandes en palabras en español
def formatear_numero(valor):
    if valor >= 1_000_000_000_000:
        return f'{valor / 1_000_000_000:.2f} billones'
    elif valor >= 1_000_000_000:
        return f'{valor / 1_000_000_000:.0f} mil millones'  # Mil millones
    elif valor >= 1_000_000:
        return f'{valor / 1_000_000:.0f} millones'  # Millones
    elif valor >= 1_000:
        return f'{valor / 1_000:.0f} mil'  # Mil
    else:
        return str(valor)
# Configurar los valores y el texto del eje Y
tickvals = [2500000000,3000000000,3200000000]
ticktext = [formatear_numero(val) for val in tickvals]

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=año,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for a, p in zip(año, promedio):
    fig.add_annotation(
        x=a,
        y=p,
        text=f"${p:,.2f}",  # Formatear el valor del promedio como moneda
        showarrow=False,
        font=dict(color='black', size=12),
        xshift=0,
        yshift=17,
        textangle=0
    )
# Actualizar diseño
fig.update_layout(
    #title='Plusvalía estimada',   
    yaxis=dict(
        title="Monto",
        gridwidth=1,  # Ancho de las líneas de la cuadrícula
        gridcolor='#dddcda', 

    ),
    xaxis=dict(
        gridcolor='#dddcda', 
        tickmode='array',
        tickvals=año,
        ticktext=año
    ),
    plot_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=10, r=10, t=10, b=10)
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_plusvaliaestimada_ano', carpeta='assets/graficas')
fig.show()